# Situering
- De eigenaar van een huishoudelijke zonnepaneelinstallatie zou graag continu een voorspelling hebben van de opbrengst van zijn panelen gedurende de komende uren om het eigen verbruik te optimaliseren: bij een verwachte hoge opbrengst kan hij dan bijv. beslissen om de wasmachine aan te zetten. 
- Hij beschikt over de meterstand per uur sedert ongeveer één jaar (solar.csv). 
- Daarnaast zijn ook de gegevens van de waarnemingen van het weer (weather.csv) en 
- de uren van zonsopgang en –ondergang in dezelfde periode periode (sunrise-sunset.xlsx). 

# Vraag
- Stel een regressiemodel op om de opbrengst per uur te voorspellen. 
- Kies een optimaal regressiemodel door verschillende modellen uit te proberen en te vergelijken volgens de "best practices". 
- Kies als maatstaf de gemiddelde afwijking van de absolute waarde op uurbasis.




In [1]:
import sys
import sklearn
import numpy as np
import pandas as pd
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import datetime

## Preparation and cleanup of solar data

In [2]:
def prepare_solar():
    solar = pd.read_csv('datasets/solar.csv')  
    print(solar.head(10))
    print('-------------')
    print(solar.tail(10))
    solar['datetime'] = pd.to_datetime(solar['timestamp'].str[0:19],format='%Y-%m-%d %H:%M:%S')      
    solar['datetime_tz'] = solar['datetime'].dt.tz_localize('Europe/Berlin',ambiguous='NaT')
    solar['datetime_utc'] = solar['datetime_tz'].dt.tz_convert('UTC')
    solar['date'] = solar['datetime_utc'].dt.date
    solar['hour'] = solar['datetime_utc'].dt.hour
    solar = solar[['date','hour','kwh']]
    solar.dropna(inplace=True)
    solar.drop_duplicates(inplace=True)
    # todo timediff en alles > 1u er uit. 
    return solar

solar = prepare_solar()
solar.info()
solar.head()

                       timestamp   kwh
0  2023-03-11 16:00:10.160454+01  0.54
1  2023-03-11 17:00:10.217795+01  1.02
2  2023-03-11 18:00:10.284064+01  1.17
3  2023-03-11 19:00:10.224836+01  1.18
4  2023-03-11 20:00:10.201847+01  1.18
5  2023-03-11 21:00:10.338166+01  1.18
6  2023-03-11 22:00:10.189411+01  1.18
7   2023-03-11 23:00:10.14459+01  1.18
8  2023-03-12 00:00:10.058642+01  1.18
9  2023-03-12 01:00:10.136006+01  1.18
-------------
                          timestamp        kwh
9695  2024-04-19 01:00:10.322579+02  2417.3321
9696  2024-04-19 02:00:10.339044+02  2417.3321
9697  2024-04-19 03:00:10.343655+02  2417.3321
9698  2024-04-19 04:00:10.507106+02  2417.3321
9699  2024-04-19 05:00:10.326829+02  2417.3321
9700  2024-04-19 06:00:10.326636+02  2417.3321
9701  2024-04-19 07:00:10.424826+02  2417.3321
9702  2024-04-19 08:00:10.360449+02  2417.3336
9703  2024-04-19 09:00:10.437154+02  2417.3701
9704  2024-04-19 10:00:10.748713+02  2417.4788
<class 'pandas.core.frame.DataFrame'>
In

,date,hour,kwh
0,2023-03-11,15.0,0.54
1,2023-03-11,16.0,1.02
2,2023-03-11,17.0,1.17
3,2023-03-11,18.0,1.18
4,2023-03-11,19.0,1.18


## Preparation and cleanup of weather data

In [3]:
def prepare_weather():
    weather = pd.read_csv("datasets/weather.csv")
    print(weather.head(10))
    weather = weather[['timestamp','temp','pressure','cloudiness','humidity_relative']]
    weather = weather.groupby(by=['timestamp']).mean().reset_index()
    weather['datetime'] = pd.to_datetime(weather['timestamp'],format='%Y-%m-%dT%H:%M:%S')
    weather['date'] = weather['datetime'].dt.date
    weather['hour'] = weather['datetime'].dt.hour   
    weather = weather[['date','hour','temp','pressure','cloudiness','humidity_relative']]
    weather.drop_duplicates(inplace=True)
    return weather

weather = prepare_weather()
weather.info()
weather.head(30)


                                      FID                    the_geom  code  \
0  synop_data.6418.2023-03-01 09:00:00+00  POINT (51.347375 3.201846)  6418   
1  synop_data.6414.2023-03-01 09:00:00+00   POINT (50.90398 3.121692)  6414   
2  synop_data.6407.2023-03-01 09:00:00+00  POINT (51.200341 2.887306)  6407   
3  synop_data.6434.2023-03-01 09:00:00+00  POINT (50.980293 3.816003)  6434   
4  synop_data.6434.2023-03-01 10:00:00+00  POINT (50.980293 3.816003)  6434   
5  synop_data.6418.2023-03-01 10:00:00+00  POINT (51.347375 3.201846)  6418   
6  synop_data.6414.2023-03-01 10:00:00+00   POINT (50.90398 3.121692)  6414   
7  synop_data.6407.2023-03-01 10:00:00+00  POINT (51.200341 2.887306)  6407   
8  synop_data.6414.2023-03-01 11:00:00+00   POINT (50.90398 3.121692)  6414   
9  synop_data.6418.2023-03-01 11:00:00+00  POINT (51.347375 3.201846)  6418   

             timestamp  precip_quantity  precip_range  temp  temp_min  \
0  2023-03-01T09:00:00              NaN           NaN   4

,date,hour,temp,pressure,cloudiness,humidity_relative
0,2023-03-01,9,2.975,1029.500,0.5,66.300000
1,2023-03-01,10,4.000,1029.150,0.5,60.900000
2,2023-03-01,11,4.800,1028.575,0.5,55.000000
3,2023-03-01,12,5.375,1028.100,0.5,53.166667
4,2023-03-01,13,5.900,1027.125,0.5,51.833333
5,2023-03-01,14,5.950,1026.350,0.5,53.400000
6,2023-03-01,15,5.975,1025.625,0.5,55.366667
7,2023-03-01,16,5.700,1025.075,0.5,57.466667
8,2023-03-01,17,5.300,1024.725,0.5,58.866667
9,2023-03-01,18,4.700,1024.825,0.5,58.800000


## Preparation and cleanup of sunrise/sunset data

In [4]:
def prepare_sunrise_sunset():
    sunrise_sunset = pd.read_excel("datasets/sunrise-sunset.xlsx") 
    print(sunrise_sunset.head(10))
    sunrise_sunset.rename(columns={'datum':'date'}, inplace=True)
    sunrise_sunset['sunrise'] = [pd.Timestamp.combine(d,t) for d,t in zip(sunrise_sunset['date'],sunrise_sunset['Opkomst'])]
    sunrise_sunset['noon'] = [pd.Timestamp.combine(d,t) for d,t in zip(sunrise_sunset['date'],sunrise_sunset['Op ware middag'])]
    sunrise_sunset['sunset'] = [pd.Timestamp.combine(d,t) for d,t in zip(sunrise_sunset['date'],sunrise_sunset['Ondergang'])]
    sunrise_sunset['sunrise'] = sunrise_sunset['sunrise'].dt.tz_localize('Europe/Berlin',ambiguous='NaT')
    sunrise_sunset['sunrise'] = sunrise_sunset['sunrise'].dt.tz_convert('UTC')
    sunrise_sunset['sunset'] = sunrise_sunset['sunset'].dt.tz_localize('Europe/Berlin',ambiguous='NaT')
    sunrise_sunset['sunset'] = sunrise_sunset['sunset'].dt.tz_convert('UTC')
    sunrise_sunset['date'] = sunrise_sunset['date'].dt.date
    sunrise_sunset = sunrise_sunset[['date','sunrise','sunset']]
    return sunrise_sunset

sunrise_sunset = prepare_sunrise_sunset()
sunrise_sunset.info()
sunrise_sunset.head()

       datum   Opkomst Op ware middag Ondergang
0 2023-01-01  08:45:00       12:46:00  16:47:00
1 2023-01-02  08:45:00       12:46:00  16:48:00
2 2023-01-03  08:45:00       12:47:00  16:49:00
3 2023-01-04  08:44:00       12:47:00  16:51:00
4 2023-01-05  08:44:00       12:48:00  16:52:00
5 2023-01-06  08:44:00       12:48:00  16:53:00
6 2023-01-07  08:43:00       12:49:00  16:54:00
7 2023-01-08  08:43:00       12:49:00  16:56:00
8 2023-01-09  08:43:00       12:50:00  16:57:00
9 2023-01-10  08:42:00       12:50:00  16:58:00
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype              
---  ------   --------------  -----              
 0   date     731 non-null    object             
 1   sunrise  731 non-null    datetime64[ns, UTC]
 2   sunset   731 non-null    datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](2), object(1)
memory usage: 17.3+ KB


,date,sunrise,sunset
0,2023-01-01,2023-01-01 07:45:00+00:00,2023-01-01 15:47:00+00:00
1,2023-01-02,2023-01-02 07:45:00+00:00,2023-01-02 15:48:00+00:00
2,2023-01-03,2023-01-03 07:45:00+00:00,2023-01-03 15:49:00+00:00
3,2023-01-04,2023-01-04 07:44:00+00:00,2023-01-04 15:51:00+00:00
4,2023-01-05,2023-01-05 07:44:00+00:00,2023-01-05 15:52:00+00:00


Combine solar and weather data in a single dataframe. 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9340 entries, 0 to 9339
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               9340 non-null   object 
 1   hour               9340 non-null   float64
 2   kwh                9340 non-null   float64
 3   temp               9340 non-null   float64
 4   pressure           9340 non-null   float64
 5   cloudiness         9340 non-null   float64
 6   humidity_relative  9340 non-null   float64
dtypes: float64(6), object(1)
memory usage: 510.9+ KB


,date,hour,kwh,temp,pressure,cloudiness,humidity_relative
0,2023-03-11,15.0,0.54,6.85,1012.025,6.0,54.833333
1,2023-03-11,16.0,1.02,6.30,1011.625,7.0,57.800000
2,2023-03-11,17.0,1.17,5.95,1011.500,7.0,60.366667
3,2023-03-11,18.0,1.18,5.25,1011.850,7.0,64.500000
4,2023-03-11,19.0,1.18,4.85,1011.650,7.0,68.633333


Now also combine this dataset with sunrise_sunset. 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9340 entries, 0 to 9339
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   date               9340 non-null   object             
 1   hour               9340 non-null   float64            
 2   kwh                9340 non-null   float64            
 3   temp               9340 non-null   float64            
 4   pressure           9340 non-null   float64            
 5   cloudiness         9340 non-null   float64            
 6   humidity_relative  9340 non-null   float64            
 7   sunrise            9340 non-null   datetime64[ns, UTC]
 8   sunset             9340 non-null   datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](2), float64(6), object(1)
memory usage: 656.8+ KB


,date,hour,kwh,temp,pressure,cloudiness,humidity_relative,sunrise,sunset
0,2023-03-11,15.0,0.54,6.85,1012.025,6.0,54.833333,2023-03-11 06:06:00+00:00,2023-03-11 17:40:00+00:00
1,2023-03-11,16.0,1.02,6.30,1011.625,7.0,57.800000,2023-03-11 06:06:00+00:00,2023-03-11 17:40:00+00:00
2,2023-03-11,17.0,1.17,5.95,1011.500,7.0,60.366667,2023-03-11 06:06:00+00:00,2023-03-11 17:40:00+00:00
3,2023-03-11,18.0,1.18,5.25,1011.850,7.0,64.500000,2023-03-11 06:06:00+00:00,2023-03-11 17:40:00+00:00
4,2023-03-11,19.0,1.18,4.85,1011.650,7.0,68.633333,2023-03-11 06:06:00+00:00,2023-03-11 17:40:00+00:00


## Feature Engineering
Only keep following features: 
- dayinyear: number of the day in de year (1/1 = 1, 31/12 = 365)
- sunrise_delta: hours after sunrise
- sunset_delta: hours before sunset
- temp
- pressure
- cloudiness
- humidity
- production (kW): yield of the current hour

<class 'pandas.core.frame.DataFrame'>
Index: 9337 entries, 1 to 9339
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   dayinyear          9337 non-null   int64  
 1   sunrise_delta      9337 non-null   int64  
 2   sunset_delta       9337 non-null   int64  
 3   temp               9337 non-null   float64
 4   pressure           9337 non-null   float64
 5   cloudiness         9337 non-null   float64
 6   humidity_relative  9337 non-null   float64
 7   production         9337 non-null   float64
dtypes: float64(5), int64(3)
memory usage: 656.5 KB
    dayinyear  sunrise_delta  sunset_delta    temp  pressure  cloudiness  \
1          70             10             1   6.300  1011.625         7.0   
2          70             11             0   5.950  1011.500         7.0   
3          70             12            -1   5.250  1011.850         7.0   
4          70             13            -2   4.850  1011.650   

Store the current dataframe to a csv file so we can use it later. 

Read the data from the csv file

Split the dataset in a training and a testset

Create a Random Forest model to predict the hourly production. 
- Create a pipeline with a StandardScaler and and a random forest regressor
- Find the optimal parameter combination amongst
  - bootstrap: False, True
  - n_estimators: 50 - 200 with steps of 50
  - max_depth: 10 - 50 with steps of 10 

Fitting 4 folds for each of 24 candidates, totalling 96 fits
{'randomforestregressor__bootstrap': True, 'randomforestregressor__max_depth': np.int64(20), 'randomforestregressor__n_estimators': np.int64(100)}
-0.07625617610214974


Determine the mean absolute error on the test set. Is this a useful model? 

MAE = 0.064 kWh


Explain the concept of noise in this context

.

Store the model to a file. 

['solar.pkl']